<a href="https://colab.research.google.com/github/nayun12/Tobigs_17/blob/main/week5_SVM_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

In [4]:
X_train

array([[ 0.78522493,  0.32015325,  0.77221097,  1.04726529],
       [-0.26563371, -1.29989934,  0.0982814 , -0.11996537],
       [ 0.43493872,  0.78302542,  0.94069336,  1.43634218],
       [-0.84944407,  0.78302542, -1.24957775, -1.28719604],
       [-0.38239578, -1.7627715 ,  0.15444219,  0.13941922],
       [ 0.55170079, -0.374155  ,  1.05301496,  0.7878807 ],
       [ 0.31817664, -0.14271892,  0.65988937,  0.7878807 ],
       [ 0.20141457, -0.374155  ,  0.43524618,  0.39880381],
       [-1.66677857, -0.14271892, -1.36189934, -1.28719604],
       [-0.14887164, -0.60559109,  0.21060299,  0.13941922],
       [-0.14887164, -1.06846325, -0.12636179, -0.24965767],
       [ 0.31817664, -0.60559109,  0.15444219,  0.13941922],
       [ 0.66846286, -0.83702717,  0.88453256,  0.91757299],
       [ 0.0846525 , -0.14271892,  0.77221097,  0.7878807 ],
       [-0.49915786, -0.14271892,  0.43524618,  0.39880381],
       [-0.26563371, -0.60559109,  0.65988937,  1.04726529],
       [ 2.18636979,  1.

In [5]:
X_test

array([[-0.14887164, -0.374155  ,  0.26676379,  0.13941922],
       [ 0.31817664, -0.60559109,  0.54756778,  0.00972692],
       [ 0.31817664, -1.06846325,  1.05301496,  0.26911151],
       [-1.5500165 , -1.7627715 , -1.36189934, -1.15750374],
       [ 0.0846525 ,  0.32015325,  0.60372857,  0.7878807 ],
       [ 0.78522493, -0.14271892,  0.99685416,  0.7878807 ],
       [-0.84944407,  1.70876975, -1.24957775, -1.15750374],
       [ 0.20141457, -0.14271892,  0.60372857,  0.7878807 ],
       [-0.38239578,  2.63451409, -1.30573855, -1.28719604],
       [-0.38239578, -1.29989934,  0.15444219,  0.13941922],
       [ 0.66846286,  0.08871717,  0.99685416,  0.7878807 ],
       [-0.38239578,  1.0144615 , -1.36189934, -1.28719604],
       [-0.49915786,  0.78302542, -1.13725615, -1.28719604],
       [ 0.43493872, -0.60559109,  0.60372857,  0.7878807 ],
       [ 0.55170079, -1.7627715 ,  0.37908538,  0.13941922],
       [ 0.55170079,  0.55158933,  0.54756778,  0.52849611],
       [-1.19973028,  0.

In [6]:
class SVM_OVR:
    def __init__(self, num_classes, kernel, C, gamma):
        self.num_classes = num_classes
        self.clfs = [SVC(kernel = kernel, C = C, gamma = gamma) for _ in range(num_classes)]
        self.classes = None
        
    def fit(self, X_train, y_train):
        y_train = pd.get_dummies(y_train)
        for i in range(self.num_classes):
            self.clfs[i].fit(X_train,y_train.iloc[:,i]) 
            # 각 클래스별로 인지 아닌지를 판단하는 분류기를 학습시킵니다.
        self.classes = y_train.columns
    
    def predict(self, X_test):
        pred_df = pd.DataFrame([svm.predict(X_test) for svm in self.clfs]).T # 각 클래스 별 예측값
        decisions = np.array([svm.decision_function(X_test) for svm in self.clfs]).T # 각 클래스 별 거리
        
        final_pred = []
        for i in range(len(pred_df)):
            if sum(pred_df.iloc[i]) == 1:
                label = pred_df.iloc[i][pred_df.iloc[i] == 1].index[0]
                final_pred.append(self.classes[label])            
            else:
                label = np.argmax(decisions[i])
                final_pred.append(self.classes[label])
        
        return final_pred

In [7]:
clf = SVM_OVR(num_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [8]:
for gt, pr in zip(y_test, pred):
    print('%s%20s'%(gt, pr))

versicolor          versicolor
versicolor          versicolor
virginica          versicolor
setosa           virginica
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
setosa              setosa
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
setosa              setosa
versicolor          versicolor
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
virginica           virginica
versicolor          versicolor


In [9]:
accuracy_score(y_test, pred)

0.8666666666666667

In [10]:
from itertools import combinations

In [11]:
class SVM_OVO:
    def __init__(self, num_classes, kernel, C, gamma):
        self.num_classes = num_classes
        self.clfs = [{'class' : None,'clf' : SVC(kernel = kernel, C = C, gamma = gamma)} for _ in range(int(num_classes * (num_classes-1) / 2))]
        self.classes = None
        self.combi = []
        
    def fit(self, X_train, y_train):
        self.classes = y_train.unique()
        i = 0
        for c in combinations(self.classes, 2):
            idx = (y_train == c[0]) | (y_train == c[1])
            self.clfs[i]['clf'].fit(X_train[idx], y_train[idx])
            self.clfs[i]['class'] = c
            self.combi.append(c)
            i += 1
            
    def predict(self, X_test):
        preds_df = pd.DataFrame([svm['clf'].predict(X_test) for svm in self.clfs]).T # 각 조합 별 예측
        decisions = pd.DataFrame([svm['clf'].decision_function(X_test) for svm in self.clfs]).T # 각 클래스 별 거리
        decisions.columns = self.combi
        
        final_pred = []
        for i in range(len(preds_df)):
            if preds_df.iloc[i].value_counts().iloc[0] > preds_df.iloc[i].value_counts().iloc[1]:
                label = (preds_df.iloc[i].value_counts() / len(preds_df.iloc[i])).index[0]
                final_pred.append(label)
            else:
                decision_for_row = {key : 0 for key in classes}
                for c, d in zip(decisions.iloc[i].index, decisions.iloc[i]):
                    if d > 0:
                        decision_for_row[c[0]] += d
                    else:
                        decision_for_row[c[1]] -= d
                label = sorted(decision_for_row.items(), key = lambda x : x[1], reverse = True)[0][0]
                final_pred.append(label)
        return final_pred

In [12]:
clf = SVM_OVO(num_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [13]:
for gt, pr in zip(y_test, pred):
    print('%s%20s'%(gt, pr))

versicolor          versicolor
versicolor          versicolor
virginica          versicolor
setosa           virginica
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
setosa              setosa
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
setosa              setosa
versicolor          versicolor
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
virginica           virginica
versicolor          versicolor


In [14]:
accuracy_score(y_test, pred)

0.8666666666666667

In [15]:
# 라이브러리 활용
svm = SVC(kernel ='rbf', C = 5, gamma = 5)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

accuracy_score(y_test,y_pred)

0.8666666666666667